In [ ]:
import autograd.numpy as np
import capytaine as cpy
import matplotlib.pyplot as plt
import datetime

# wecopttool
import wecopttool as wot


wb = wot.geom.WaveBot()  # use standard dimensions
mesh_size_factor = 0.5 # 1.0 for default, smaller to refine mesh
mesh = wb.mesh(mesh_size_factor)
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")
fb.add_translation_dof(name="Heave")
ndof = fb.nb_dofs

stiffness = wot.hydrostatics.stiffness_matrix(fb).values
mass = wot.hydrostatics.inertia_matrix(fb).values

f1 = 0.05
nfreq = 50
freq = wot.frequency(f1, nfreq, False) # False -> no zero frequency

bem_data = wot.run_bem(fb, freq)

name = ["PTO_Heave",]
kinematics = np.eye(ndof)
controller = None
efficiency = None
pto_impedance = None
pto = wot.pto.PTO(ndof, kinematics, controller, pto_impedance, efficiency, name)

# PTO dynamics forcing function
f_add = {'PTO': pto.force_on_wec}

# Constraint
f_max = 2000.0
nsubsteps = 4

def const_f_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    f = pto.force_on_wec(wec, x_wec, x_opt, nsubsteps)
    return f_max - np.abs(f.flatten())

ineq_cons = {'type': 'ineq',
             'fun': const_f_pto,
             }
constraints =  [ineq_cons]

wec = wot.WEC.from_bem(bem_data,
                    inertia_matrix=mass,
                    hydrostatic_stiffness=stiffness,
                    constraints=constraints,
                    friction=None,
                    f_add=f_add,
                    )

amplitude = 0.0625  
wavefreq = 0.3
phase = 30
wavedir = 0
waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

obj_fun = pto.mechanical_average_power
nstate_opt = 2*nfreq+1

scale_x_wec = 1e1
scale_x_opt = 1e-3
scale_obj = 1e-2

options = {'maxiter': 1000, 'ftol': 1e-8}
results = wec.solve(
    waves, 
    obj_fun, 
    nstate_opt,
    optim_options=options,
    scale_obj=scale_obj, scale_x_wec=scale_x_wec, scale_x_opt=scale_x_opt, 
    )
opt_mechanical_average_power = results.fun
print(f'Optimal average mechanical power: {opt_mechanical_average_power} W')

wec_fdom, wec_tdom = wec.post_process(results, waves)
pto_fdom, pto_tdom = pto.post_process(wec, results, waves)

In [ ]:
for var in ['pos', 'vel', 'acc']:
    fig, ax = plt.subplots()
    wec_tdom[var].plot(ax=ax, label="wec")
    pto_tdom[var].plot(ax=ax, label="pto", linestyle='--')
    plt.legend()

In [ ]:
fig, ax = plt.subplots()
wec_tdom['force'].sel(type='PTO').plot(ax=ax, label="wec")
pto_tdom['force'].plot(ax=ax, label="pto", linestyle='--')
plt.legend()